### Zona para importar

In [67]:
#imports
import pandas as pd
import numpy as np
import re
import matplotlib as mp
import reverse_geocoder as rg
import math

-------
### Carga de datos y comprobación

In [68]:
#Cargamos el archivo
data_location = './data/properatti.csv'
dataset = pd.read_csv(data_location)
dataset.head(7)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,...,78.0,1226.415094,1666.666667,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...


In [69]:
#Verificamos columnas
dataset.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

-------
#### Normalización de los nulos para los precios

In [70]:
#Vemos nulos de los campos precio y precio en dólares
price_nulls = dataset.price.isnull().sum()
price_aprx_usd_nulls = dataset.price_aprox_usd.isnull().sum()
print(price_nulls)
print(price_aprx_usd_nulls)

20410
20410


In [71]:
#Creo una columna de precios normalizados usando el dólar como moneda
dataset['price_normalized'] = dataset.apply(lambda x: x.price if x.currency == "USD" else x.price_aprox_usd,axis=1)

In [72]:
#Veo los nulos que quedan
dataset.loc[dataset.price_aprox_usd.isnull(),['price','currency','price_aprox_local_currency','price_aprox_usd','price_normalized']].head(10)

,price,currency,price_aprox_local_currency,price_aprox_usd,price_normalized
5,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN
76,NaN,NaN,NaN,NaN,NaN
83,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN
85,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN


In [73]:
#Drop precios en dolares nulos
#Valores antes
print("Precio normalizado nulos: ", dataset.price_normalized.isnull().sum())
print("Precio normalizado no nulos: ",dataset.price_normalized.notnull().sum())
print("Tamaño del dataset: ", dataset.shape)
print("-----")
#Dropeo
dataset.dropna(subset=['price_normalized'],inplace=True)
#Valores después
print("Precio normalizado nulos: ", dataset.price_normalized.isnull().sum())
print("Precio normalizado no nulos: ", dataset.price_normalized.notnull().sum())
print("Tamaño del dataset: ", dataset.shape)

Precio normalizado nulos:  20410
Precio normalizado no nulos:  100810
Tamaño del dataset:  (121220, 27)
-----
Precio normalizado nulos:  0
Precio normalizado no nulos:  100810
Tamaño del dataset:  (100810, 27)


---
#### Normalización de los place_nam

In [74]:
#Veo cuántos nulos tiene la columna place_name
dataset.place_name.isnull().sum()

23

* Método Alejandro (con fórmula)

In [ ]:
#Convierto esos NaN al valor que tiene en la columna place_with_parent_names
dataset.loc[dataset.place_name.isnull(),'place_name'] = dataset.loc[dataset.place_name.isnull()].place_with_parent_names.str.split('|', expand=True)[3]

* Método Ignacio (con regex)

In [75]:
#Convierto los NaN al valor que tiene en la columna place_with_parent_names usando regex
patron_placename='\|(?P<nombre>[A-Za-z]+)\|\|'
patron_placename_regex = re.compile(patron_placename)
place_name_null = dataset.loc[dataset.place_name.isnull()]
dataset.loc[dataset.place_name.isnull(),'place_name'] = place_name_null \
                                                            .place_with_parent_names \
                                                            .apply(lambda x: patron_placename_regex.search(x).group('nombre'))
#Cuántos nulos quedaron
dataset.place_name.isnull().sum()

0

------

In [76]:
#Relleno manualmente un dato que está suelto
mask=(dataset.place_name=='coordenadas 34.255511')
dataset.loc[mask,'place_name'] = 'Rincón de Milberg'

---
#### Achico el dataset a valores de Capital Federal solamente

In [77]:
#Me quedo solo con barrios de capital
#1. Copio los barrios y los Palermos los convierto a Palermo
dataset['barrios_normalized'] = dataset.place_name.apply(lambda x: "Palermo" if "Palermo" in x else x)

In [78]:
#2. Uso una libreria externa para completar los barrios que sean Capital Federal con el barrio verdadero usando lat-long
mask=(dataset['place_name']=='Capital Federal') & (dataset['lat'].notnull())
dataset.loc[mask,'barrios_normalized']
dataset.loc[mask,'barrios_normalized']=dataset.loc[mask].apply(lambda x: rg.search((x.lat,x.lon))[0].get('name'),axis=1)

In [79]:
#3. Filtro solo barrios de capital federal
barrios_capfed = ['Agronomía','Almagro','Balvanera','Barracas','Belgrano','Boedo','Caballito','Chacarita','Coghlan','Colegiales','Constitución','Flores','Floresta','La Boca','La Paternal','Liniers','Mataderos','Monte Castro','Montserrat','Nueva Pompeya','Nuñez','Palermo','Parque Avellaneda','Parque Chacabuco','Parque Chas','Parque Patricios','Puerto Madero','Recoleta','Retiro','Saavedra','San Cristóbal','San Nicolás','San Telmo','Versalles','Villa Crespo','Villa Devoto','Villa General Mitre','Villa Lugano','Villa Luro','Villa Ortúzar','Villa Pueyrredón','Villa Real','Villa Riachuelo','Villa Santa Rita','Villa Soldati','Villa Urquiza','Villa del Parque','Vélez Sarsfield']
data_capfed = dataset[dataset['barrios_normalized'].isin(barrios_capfed)]

---
### Completar los null para los ambientes (rooms)

In [80]:
#Máscara para obtener los nulos en rooms
mask_rooms_nulls = data_capfed.rooms.isnull()
print('Rooms empty antes: ',data_capfed.rooms.isnull().sum())

Rooms empty antes:  12271


In [81]:
#Relleno con patron regex segun la palabra ambiente
patron = '((?P<numero>\d|mono|MONO|Mono|dos|DOS|Dos|tres|TRES|Tres|cuatro|CUATRO|Cuatro|cinco|CINCO|Cinco|seis|SEIS|Seis)\s*(?P<ambiente>amb|AMB|Amb))'
patron_regex = re.compile(patron)

def convertRoom(row):
    try:
        if isinstance(row['description'],str):
            resultado = patron_regex.search(row['description'])
            return resultado.group('numero')
    except:
        return np.nan
        
data_capfed.loc[mask_rooms_nulls,'rooms'] = data_capfed.loc[mask_rooms_nulls].apply(convertRoom,axis=1)
print('Rooms empty después: ',data_capfed.loc[:,"rooms"].isnull().sum())

Rooms empty después:  4271


C:\Users\nacho\miniconda3\envs\dsdh22_test\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [84]:
#Relleno con patron regex segun la palabra dormitorio
patron = '((?P<numero2>\d|un|UN|Un|dos|DOS|Dos|tres|TRES|Tres|cuatro|CUATRO|Cuatro|cinco|CINCO|Cinco|seis|SEIS|Seis)(\s\w+)?\s(?P<dormitorios>dorm|Dorm|DORM|hab|HAB|Hab))'
patron_regex = re.compile(patron)

def convertRoom(row):
    if isinstance(row['description'],str):
        resultado = patron_regex.finditer(row['description'])
        dormitorios=0
        for m in resultado:
            if m.group('numero2') is None:
                dormitorios=np.nan
            else:
                if(m.group('numero2').lower()=='un'):
                    dormitorios+=1
                elif(m.group('numero2').lower()=='dos'):
                    dormitorios+=2
                elif(m.group('numero2').lower()=='tres'):
                    dormitorios+=3
                elif(m.group('numero2').lower()=='cuatro'):
                    dormitorios+=4
                elif(m.group('numero2').lower()=='cinco'):
                    dormitorios+=5
                elif(m.group('numero2').lower()=='seis'):
                    dormitorios+=6
                else:
                    dormitorios+=int(m.group('numero2'))
        if not(np.isnan(dormitorios)) and dormitorios>0:
            return dormitorios
        else:
            return np.nan

mask_rooms_nulls = data_capfed.rooms.isnull()
data_capfed.loc[mask_rooms_nulls,'rooms'] = data_capfed.loc[mask_rooms_nulls].apply(convertRoom,axis=1)
print('Rooms empty después: ',data_capfed.loc[:,"rooms"].isnull().sum())

Rooms empty después:  2994


In [85]:
#Completo los store con rooms = 1
mask_nulls = data_capfed.property_type=='store'
data_capfed.loc[mask_nulls,'rooms'] = 1
print('Rooms empty: ',data_capfed.loc[:,"rooms"].isnull().sum())

Rooms empty:  2270


C:\Users\nacho\miniconda3\envs\dsdh22_test\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


---
#### Completar los null para los m2 (surface_total_in_m2)

In [86]:
#Se completan los surface_total_in_m2 null con los surface_covered_in_m2 que existan
mask=data_capfed.surface_total_in_m2.isnull()
surface_null=data_capfed.loc[mask]
print("Surface null antes: ",surface_null.surface_total_in_m2.isnull().sum())
data_capfed.loc[mask,"surface_total_in_m2"] = \
                                surface_null.apply(lambda x: x.surface_covered_in_m2 \
                                                   if not(np.isnan(x.surface_covered_in_m2)) \
                                                   else x.surface_total_in_m2,axis=1)
print("Surface null después: ", data_capfed.surface_total_in_m2.isnull().sum())

Surface null antes:  4776
Surface null después:  487


In [115]:
data=data_capfed.loc[data_capfed.surface_total_in_m2.isnull(),"description"]
data.iloc[19]

'EDIFICIO DANTE 59 1 AMBIENTE A ESTRENAR DEL 1 AL 9 PISO 35 M2, MUY BUENOS DETALLES DE TERMINACION PISOS DE PORCELLANATO  Aviso publicado por Pixel Inmobiliario (Servicio de Páginas Web para Inmobiliarias).'

In [88]:
data_capfed.isnull().sum()/data_capfed.shape[0]*100

Unnamed: 0                     0.000000
operation                      0.000000
property_type                  0.000000
place_name                     0.000000
place_with_parent_names        0.000000
country_name                   0.000000
state_name                     0.000000
geonames_id                    3.399813
lat-lon                       22.377026
lat                           22.377026
lon                           22.377026
price                          0.000000
currency                       0.000000
price_aprox_local_currency     0.000000
price_aprox_usd                0.000000
surface_total_in_m2            1.978147
surface_covered_in_m2          5.816646
price_usd_per_m2              19.444332
price_per_m2                   5.824769
floor                         90.068646
rooms                          9.220521
expenses                      78.484098
properati_url                  0.000000
description                    0.000000
title                          0.000000


24619

In [10]:
#Analicemos los lugares y cantidad de propiedades por lugar
dataset.pivot_table(index='place_name',columns='property_type',fill_value='',aggfunc={'property_type':len, 'price_normalized':np.std})

price_normalized                              \
property_type                         PH apartment    house    store   
place_name                                                             
 Country Maschwitz Club                              112731            
 los alamos                                                            
9 de Abril                                                   70710.7   
Abasto                            119416     60951   160903   320055   
Abril Club de Campo                                  236361            
...                                  ...       ...      ...      ...   
William Morris                                                         
Yerba Buena                                52879.9   389096   165821   
Zelaya                                              60650.4            
Zárate                                               301473            
coordenadas 34.255511                                                  

                        property_type                        
property_type                      PH apartment house store  
place_name                                                   
 Country Maschwitz Club                             3        
 los alamos                                   1              
9 de Abril                                          1     2  
Abasto                             13        77    11     8  
Abril Club de Campo                                 6        
...                               ...       ...   ...   ...  
William Morris                                      1        
Yerba Buena                                   5    19     4  
Zelaya                                              7        
Zárate                                             13        
coordenadas 34.255511                               1        

[974 rows x 8 columns]

In [147]:
dataset.groupby(['place_name','property_type'])['price_normalized'].describe()

count           mean            std  \
place_name              property_type                                        
 Country Maschwitz Club house            3.0  311666.666667  112731.243821   
 los alamos             apartment        1.0   81600.000000            NaN   
9 de Abril              house            1.0  110000.000000            NaN   
                        store            2.0  350000.000000   70710.678119   
Abasto                  PH              13.0  176353.846154  119416.418577   
...                                      ...            ...            ...   
Yerba Buena             apartment        5.0   91987.110000   52879.853546   
                        house           19.0  318982.235263  389096.033762   
                        store            4.0  106205.287500  165820.897896   
Zelaya                  house            7.0  129857.142857   60650.442624   
Zárate                  house           13.0  362538.461538  301473.496730   

                                             min          25%         50%  \
place_name              property_type                                       
 Country Maschwitz Club house          195000.00  257500.0000  320000.000   
 los alamos             apartment       81600.00   81600.0000   81600.000   
9 de Abril              house          110000.00  110000.0000  110000.000   
                        store          300000.00  325000.0000  350000.000   
Abasto                  PH              55000.00   98700.0000  160000.000   
...                                          ...          ...         ...   
Yerba Buena             apartment       17867.69   77013.6000  100000.000   
                        house          100950.61  160424.4450  196292.860   
                        store            6383.89   21365.4625   32134.425   
Zelaya                  house           35000.00  114500.0000  125000.000   
Zárate                  house           60000.00  198000.0000  255000.000   

                                             75%         max  
place_name              property_type                         
 Country Maschwitz Club house          370000.00   420000.00  
 los alamos             apartment       81600.00    81600.00  
9 de Abril              house          110000.00   110000.00  
                        store          375000.00   400000.00  
Abasto                  PH             205000.00   510000.00  
...                                          ...         ...  
Yerba Buena             apartment      100000.00   165054.26  
                        house          269201.64  1714040.85  
                        store          116974.25   354168.41  
Zelaya                  house          140000.00   240000.00  
Zárate                  house          370000.00   950000.00  

[1924 rows x 8 columns]

In [105]:
data_group = dataset.groupby(['place_name','property_type'])
data_group.size()

place_name               property_type
 Country Maschwitz Club  house             3
 los alamos              apartment         1
9 de Abril               house             1
                         store             2
Abasto                   PH               13
                                          ..
Yerba Buena              house            19
                         store             4
Zelaya                   house             7
Zárate                   house            13
coordenadas 34.255511    house             1
Length: 1925, dtype: int64

In [114]:
data_filtrada = data_group.filter(lambda x: x['property_type'].count() > 10)

In [115]:
print(data_filtrada.groupby(['place_name','property_type'])['property_type'].count().unstack().to_string())

property_type                             PH  apartment   house  store
place_name                                                            
Abasto                                  13.0       77.0    11.0    NaN
Acacias Blancas                          NaN        NaN    34.0    NaN
Acassuso                                 NaN       86.0   131.0    NaN
Adrogué                                 25.0      172.0   500.0   34.0
Agronomía                                NaN       27.0     NaN    NaN
Albanueva Barrio Cerrado                 NaN       11.0    11.0    NaN
Alejandro Korn                           NaN        NaN    14.0    NaN
Almagro                                 65.0      857.0    30.0   34.0
Almirante Brown                          NaN       13.0    24.0    NaN
Alta Gracia                              NaN        NaN    14.0    NaN
Altamira                                 NaN        NaN    14.0    NaN
Avellaneda                              29.0      283.0    60.0   19.0
Azul  

In [94]:
mask = dataset.place_name=='Rosario'
datos_buscados = dataset.loc[mask]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



                                          property_type                                 
property_type                                        PH  apartment      house      store
place_name                                                                              
 Country Maschwitz Club                       No Existe  No Existe          3  No Existe
 los alamos                                   No Existe          1  No Existe  No Existe
9 de Abril                                    No Existe  No Existe          1          2
Abasto                                               13         77         11          8
Abril Club de Campo                           No Existe  No Existe          6  No Existe
Acacias Blancas                               No Existe  No Existe         34  No Existe
Acassuso                                              8         86        131          8
Achiras                                       No Existe  No Existe          1  No Existe
Adrogué              